In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [2]:
from region import get_region_by_two_bases, clean_region

In [3]:
import datetime
from utils import get_path, DATETIME_FORMAT

In [4]:
sql = """
select 	a.id app_id,
		au.id au_id,
		au.created user_registration_date,
		au.updated user_last_login,
		case when au.created at time zone 'UTC' > '2020-03-30'
				then 'registration'
				else 'already exists'
		end user_check,
		patient.id patient_id,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
		p_patient.sex, 
		p_patient.birthday patient_birthday,
		p_patient.phone patient_phone,
		p_patient.email,
		date(a.finished at time zone 'UTC') consultation_date,
		case when pc.product_id in (
            	select id from product
            	where full_name like '%ВЭБ%') then 'VEBMED'
        	else (select full_name from product where pc.product_id = product.id)
        end WEBpartition,
        --
        concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
		case when doctor.profession_name is not null 
			then doctor.profession_name
			when a.id is null then NULL
			else 'Дежурный врач'
		end doctor_profession
--		
from auth_user au
--
left join person p_patient on p_patient.user_id = au.id 
and p_patient.id in (select distinct id from patient)
--
left join patient on p_patient.id = patient.person_id
--
left join appointment a on patient.id = a.patient_id 
and a.finished at time zone 'UTC' > '2020-03-30'
--
left join promotion promo on a.promotion_id = promo.id
left join product_condition pc
   on promo.product_condition_id = pc.id
--
left join doctor on a.doctor_id = doctor.id 
left join person p_doctor on doctor.person_id = p_doctor.id 
--
where au.updated at time zone 'UTC' > '2020-03-30'


"""

In [5]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-06-01 19:34:38 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-01 19:34:38 creating ssh tunnel to 172.16.100.19 as root...
2020-06-01 19:34:39 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 42159
2020-06-01 19:34:39 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [6]:
data = connection.query(sql)

2020-06-01 19:34:39 @telemed query:
                    select 	a.id app_id,
				    		au.id au_id,
				    		au.created user_registration_date,
				    		au.updated user_last_login,
				    		case when au.created at time zone 'UTC' > '2020-03-30'
				    				then 'registration'
				    				else 'already exists'
				    		end user_check,
				    		patient.id patient_id,
				    		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    		p_patient.sex, 
				    		p_patient.birthday patient_birthday,
				    		p_patient.phone patient_phone,
				    		p_patient.email,
				    		date(a.finished at time zone 'UTC') consultation_date,
				    		case when pc.product_id in (
				                	select id from product
				                	where full_name like '%ВЭБ%') then 'VEBMED'
				            	else (select full_name from product where pc.product_id = product.id)
				            end WEBpartition,
				            --
				            concat_ws(' '

In [7]:
connection.close()

2020-06-01 19:35:12 @telemed: closing postgres connection...
2020-06-01 19:35:12 closing ssh tunnel to 172.16.100.19...


In [8]:
# connection.status()

In [9]:
df = pd.DataFrame(data).copy()

In [10]:
df[df['patient_phone'] == '']

,app_id,au_id,user_registration_date,user_last_login,user_check,patient_id,patient_fio,sex,patient_birthday,patient_phone,email,consultation_date,webpartition,doctor_fio,doctor_profession
86967,NaN,88051,2018-10-20 07:58:16.346,2020-04-04 06:13:39.144,already exists,78455.0,Муратова Ольга Игоревна,FEMALE,1977-12-29,,None,None,None,,None


In [11]:
df.head()

,app_id,au_id,user_registration_date,user_last_login,user_check,patient_id,patient_fio,sex,patient_birthday,patient_phone,email,consultation_date,webpartition,doctor_fio,doctor_profession
0,127378.0,2994708,2020-03-08 14:22:11.888,2020-05-12 09:16:32.416,already exists,2932002.0,Рудий Екатерина Павловна,FEMALE,1986-03-11,79139895925,world.star@mail.ru,2020-03-30,Подписка на 1 месяц. Детский. 290,Сыров Всеволод Владимирович,Дежурный врач
1,142510.0,2546457,2020-01-18 19:33:44.504,2020-04-17 13:33:11.142,already exists,2486556.0,Хашхожев Мурат Заурбекович,MALE,1990-01-29,79280799377,None,2020-04-13,Новый год_Ренессанс,Сыров Всеволод Владимирович,Дежурный врач
2,130846.0,26398,2018-05-09 12:22:27.966,2020-04-03 10:11:49.100,already exists,23192.0,Длинов Вячеслав Геннадьевич,MALE,1978-07-26,79170779941,dvg_kazna@mail.ru,2020-04-03,ДМС_Сбербанк,Сыров Всеволод Владимирович,Дежурный врач
3,133925.0,178373,2019-02-05 07:49:01.811,2020-06-01 11:19:21.929,already exists,152413.0,Малова Ольга Андреевна,FEMALE,1992-03-17,79685539209,olga.osinceva@yandex.ru,2020-04-06,ТМ_Согласие_факт + Психология,Сыров Всеволод Владимирович,Дежурный врач
4,127392.0,1226999,2019-08-28 08:55:03.801,2020-05-30 16:51:42.926,already exists,1199651.0,Нигматзянова Ильмира Ильсуровна,None,1996-04-24,79120121876,None,2020-03-30,ДМС_Сбербанк,Сыров Всеволод Владимирович,Дежурный врач


In [12]:
df['dirty_region'] = df['patient_phone'].fillna(100).apply(get_region_by_two_bases)

In [13]:
df['region'] = df['dirty_region'].apply(clean_region)

In [14]:
df.head(15)

,app_id,au_id,user_registration_date,user_last_login,user_check,patient_id,patient_fio,sex,patient_birthday,patient_phone,email,consultation_date,webpartition,doctor_fio,doctor_profession,dirty_region,region
0,127378.0,2994708,2020-03-08 14:22:11.888,2020-05-12 09:16:32.416,already exists,2932002.0,Рудий Екатерина Павловна,FEMALE,1986-03-11,79139895925,world.star@mail.ru,2020-03-30,Подписка на 1 месяц. Детский. 290,Сыров Всеволод Владимирович,Дежурный врач,Новосибирская обл.,Новосибирская область
1,142510.0,2546457,2020-01-18 19:33:44.504,2020-04-17 13:33:11.142,already exists,2486556.0,Хашхожев Мурат Заурбекович,MALE,1990-01-29,79280799377,None,2020-04-13,Новый год_Ренессанс,Сыров Всеволод Владимирович,Дежурный врач,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика
2,130846.0,26398,2018-05-09 12:22:27.966,2020-04-03 10:11:49.100,already exists,23192.0,Длинов Вячеслав Геннадьевич,MALE,1978-07-26,79170779941,dvg_kazna@mail.ru,2020-04-03,ДМС_Сбербанк,Сыров Всеволод Владимирович,Дежурный врач,Чувашская Республика,Чувашская Республика
3,133925.0,178373,2019-02-05 07:49:01.811,2020-06-01 11:19:21.929,already exists,152413.0,Малова Ольга Андреевна,FEMALE,1992-03-17,79685539209,olga.osinceva@yandex.ru,2020-04-06,ТМ_Согласие_факт + Психология,Сыров Всеволод Владимирович,Дежурный врач,г. Москва и Московская область,Москва и Московская область
4,127392.0,1226999,2019-08-28 08:55:03.801,2020-05-30 16:51:42.926,already exists,1199651.0,Нигматзянова Ильмира Ильсуровна,None,1996-04-24,79120121876,None,2020-03-30,ДМС_Сбербанк,Сыров Всеволод Владимирович,Дежурный врач,Удмуртская Республика,Удмуртская Республика
5,136382.0,73123,2018-09-26 06:14:07.933,2020-05-31 09:34:26.203,already exists,64057.0,Ливенцов Игорь Михайлович,MALE,1983-10-14,79387001414,ya.igo007@yandex.ru,2020-04-08,РГС Телемедицина ФЛ,Сыров Всеволод Владимирович,Дежурный врач,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика
6,140420.0,3095112,2020-04-01 15:19:41.390,2020-05-26 15:19:24.624,registration,3031789.0,Бостынец Галия Саяровна,FEMALE,1976-06-06,79166928162,galiya_boss@mail.ru,2020-04-11,VEBMED,Сыров Всеволод Владимирович,Дежурный врач,Московская область,Москва и Московская область
7,160201.0,178373,2019-02-05 07:49:01.811,2020-06-01 11:19:21.929,already exists,152413.0,Малова Ольга Андреевна,FEMALE,1992-03-17,79685539209,olga.osinceva@yandex.ru,2020-04-24,ТМ_Согласие_факт + Психология,Сыров Всеволод Владимирович,Дежурный врач,г. Москва и Московская область,Москва и Московская область
8,155803.0,1739242,2019-12-05 15:28:59.227,2020-06-01 14:48:04.054,already exists,1704829.0,Сергеева Елена Владимировна,FEMALE,1996-06-03,79376763930,None,2020-04-23,Доктор рядом и Ко,Гиоев Александр Заурбекович,Дежурный врач,Москва,Москва и Московская область
9,155581.0,3568532,2020-04-21 08:07:47.440,2020-05-20 02:56:59.051,registration,3504224.0,карасева ксения,FEMALE,1986-10-23,79140547698,,2020-04-22,VEBMED,Сыров Всеволод Владимирович,Дежурный врач,Республика Бурятия,Республика Бурятия


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942210 entries, 0 to 942209
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   app_id                  92821 non-null   float64       
 1   au_id                   942210 non-null  int64         
 2   user_registration_date  942210 non-null  datetime64[ns]
 3   user_last_login         942210 non-null  datetime64[ns]
 4   user_check              942210 non-null  object        
 5   patient_id              901336 non-null  float64       
 6   patient_fio             942210 non-null  object        
 7   sex                     197848 non-null  object        
 8   patient_birthday        901341 non-null  object        
 9   patient_phone           902157 non-null  object        
 10  email                   67178 non-null   object        
 11  consultation_date       92821 non-null   object        
 12  webpartition            86096 

In [16]:
df.to_excel('User-Login.xlsx', index=False)